# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [104]:
# import numpy and pandas
import pandas as pd
import numpy as np

from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest


from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
from scipy.stats import sem

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [106]:
# Run this code:
salaries = pd.read_csv('/Users/tiagovhp/Ironhack/Week_4/Day_2/lab-hypothesis-testing-en/data/Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [109]:
# Your code here
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [112]:
# First, let's convert into a pandas Dataframe
salaries = pd.DataFrame(salaries)


# Fiter the Dataset to obtain only data from those that have Hourly Rate
salaries_hourly = salaries[salaries['Salary or Hourly'] == 'Hourly']

# H0: Null hypoteses: The mean hourly wage of all hourly workers is 30 (niu =30)
niu = 30
# H1: Alternative hypotheses: The mean hourly wage of all hourly workers is not 30 (niu != 30)
# significance level: 
alpha = 0.05 #(confidence level of 95 %)

# Perform the One Sample T-Test
t_statistic, p_value = stats.ttest_1samp(list(salaries_hourly['Hourly Rate']),niu)

# Output results
print("T-statistic:", t_statistic)
print("P-value:", p_value)

# Step 7: Decision based on p-value
if p_value < alpha:
    print("Reject the null hypothesis: ...")
else:
    print("Fail to reject the null hypothesis: ...")

T-statistic: 20.6198057854942
P-value: 4.3230240486229894e-92
Reject the null hypothesis: ...


# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [126]:
import numpy as np
import scipy.stats as stats

# Assuming `hourly_wages` is a list or array of hourly wage data
# Replace with your actual data
hourly_wages = [/* your data here */]

# Step 1: Calculate the mean of the sample
mean_wage = np.mean(hourly_wages)

# Step 2: Calculate the standard error
standard_error = stats.sem(hourly_wages)

# Step 3: Calculate the confidence interval
confidence_interval = stats.t.interval(
    0.95,  # confidence level
    len(hourly_wages) - 1,  # degrees of freedom
    loc=mean_wage,  # mean of the sample
    scale=standard_error  # standard error
)

print("95% Confidence Interval:", confidence_interval)


32.78855771628024
12.11257268427681
8022


# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [ ]:
# Your code here
from statsmodels.stats.proportion import proportions_ztest

# Define the count of hourly workers and the total number of workers
count = # Number of hourly workers
nobs = # Total number of workers
value = 0.25  # Hypothesized proportion

# Perform the z-test
stat, p_value = proportions_ztest(count, nobs, value)

# Display the results
print(f"Z-test statistic: {stat}")
print(f"P-value: {p_value}")

# Determine if we reject the null hypothesis
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: The proportion of hourly workers is significantly different from 25%.")
else:
    print("Fail to reject the null hypothesis: The proportion of hourly workers is not significantly different from 25%.")
